<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучу модель классифицировать комментарии на позитивные и негативные. В моем распоряжении набор данных с разметкой о токсичности правок.



**Задачи**

1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Выбрать лучшую модель.



## Подготовка

In [1]:
!pip install pymystem3
!pip3 install catboost
!pip install lightgbm
!pip install scikit-learn==1.1.3

In [38]:
import pandas as pd
from pymystem3 import Mystem
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import  make_scorer
import lightgbm as lgb
from sklearn.dummy import DummyClassifier
import re 
from scipy.sparse import vstack



In [2]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\iyaro\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iyaro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\iyaro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\iyaro\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\iyaro\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
RS=12345

In [4]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv', index_col='Unnamed: 0')
except:
    data = pd.read_csv(r'C:\Users\iyaro\Downloads\toxic_comments.csv', index_col='Unnamed: 0')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [6]:
data['text'] = data['text'].apply(str)


In [7]:
data.head(5)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [8]:
stop_words = set(stopwords.words('english'))

In [9]:
%%time
lemmatizer = WordNetLemmatizer()
regex_alpha = re.compile('[^a-zA-Z]+')
def lemma(text):
    token_pos_tuples = nltk.pos_tag([
        token.lower() for token in re.sub(regex_alpha, ' ', text).split()
        if token not in stop_words
    ])
    lemmas = [
        lemmatizer.lemmatize(token, pos='n')
        if pos.startswith('N')
        else lemmatizer.lemmatize(token, pos='v')
        if pos.startswith('V')
        else lemmatizer.lemmatize(token, pos='a')
        if pos.startswith('J')
        else lemmatizer.lemmatize(token, pos='r')
        if pos.startswith('R')
        else lemmatizer.lemmatize(token)
        for token, pos in token_pos_tuples
    ]
    return ' '.join(lemmas)
data['text_lemmatized'] = data['text'].apply(lambda x: lemma(x))

Wall time: 4min 44s


In [10]:
data.head(20)

,text,toxic,text_lemmatized
0,Explanation\nWhy the edits made under my usern...,0,explanation why edits make username hardcore m...
1,D'aww! He matches this background colour I'm s...,0,d aww he match background colour i seemingly s...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i really try edit war it guy constantl...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i make real suggestion improvement i wond...
4,"You, sir, are my hero. Any chance you remember...",0,you sir hero any chance remember page
5,"""\n\nCongratulations from me as well, use the ...",0,congratulation well use tool well talk
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,cocksucker before you piss around on my work
7,Your vandalism to the Matt Shirvington article...,0,your vandalism matt shirvington article revert...
8,Sorry if the word 'nonsense' was offensive to ...,0,sorry word nonsense offensive anyway i intend ...
9,alignment on this subject and which are contra...,0,alignment subject contrary dulithgow


In [11]:
x = data.drop('toxic', axis=1)
y = data['toxic']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, stratify=y, random_state=RS)
x_test, x_valid, y_test, y_valid = train_test_split(x_test, y_test, test_size=0.5, stratify=y_test, random_state=RS)


In [12]:
vectorizer = TfidfVectorizer(stop_words=stop_words)
train_tfidf = vectorizer.fit_transform(x_train['text_lemmatized'])
valid_tfidf = vectorizer.transform(x_valid['text_lemmatized'])
test_tfidf = vectorizer.transform(x_test['text_lemmatized'])

# Вывод:
Данные загружены, текст лемматизирован, выборки поделены и применен TF-IDF. Можно перейти к обучению.

## Обучение

Обучу 4 модели. Catboost, lgb, случайный лес и логистическую регрессию.

In [34]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79646 entries, 106251 to 55812
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             79646 non-null  object
 1   text_lemmatized  79646 non-null  object
dtypes: object(2)
memory usage: 1.8+ MB


In [42]:
%%time
pipeline = Pipeline([
    ('vect', TfidfVectorizer()),
    ('model', LogisticRegression(random_state=RS))
])

parameters = {
    'vect__max_df': (0.6, 0.8, 1.0),
    'vect__min_df': (0, 1, 2, 5),
    'vect__stop_words': ('english', list(stop_words)),
    'vect__ngram_range': ((1, 1), (1, 2)),
    'model__solver': ['lbfgs', 'newton-cg', 'liblinear'],
    'model__C' : np.arange(1, 11, 2),
    'model__warm_start': [True, False],
    'model__max_iter': range(100, 500),
    'model__class_weight': [None, 'balanced']
}

pipeline_rscv = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=parameters,
    n_iter=10,
    cv=5,
    random_state=RS,
    scoring='f1'
)
pipeline_rscv.fit(x_train['text_lemmatized'], y_train)

Wall time: 7min 12s


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                             ('model',
                                              LogisticRegression(random_state=12345))]),
                   param_distributions={'model__C': array([1, 3, 5, 7, 9]),
                                        'model__class_weight': [None,
                                                                'balanced'],
                                        'model__max_iter': range(100, 500),
                                        'model__solver': ['lbfgs', 'newton-cg',
                                                          'liblinear'],
                                        'model__warm_start': [True, False],
                                        'vect__max_df':...0.8, 1.0),
                                        'vect__min_df': (0, 1, 2, 5),
                                        'vect__ngram_range': ((1, 1), (1, 2)),
                                        'vect__stop_words': ('english',
                                                             ['o', 'you', 'we',
                                                              'yourself', 'and',
                                                              'between', 'ma',
                                                              'because', 'it',
                                                              'had', 'shouldn',
                                                              'my', 'after',
                                                              'y', "don't",
                                                              'yours',
                                                              'against',
                                                              'herself', 'at',
                                                              'ours', 'her',
                                                              'has', 'while',
                                                              'their', 'any',
                                                              'that', "didn't",
                                                              'then', 'if',
                                                              'your', ...])},
                   random_state=12345, scoring='f1')

In [43]:
best_params = pipeline_rscv.best_params_
best_score = pipeline_rscv.best_score_

print("Лучшие параметры:", best_params)
print("Лучшая оценка:", best_score)

Лучшие параметры: {'vect__stop_words': 'english', 'vect__ngram_range': (1, 2), 'vect__min_df': 0, 'vect__max_df': 0.6, 'model__warm_start': False, 'model__solver': 'lbfgs', 'model__max_iter': 286, 'model__class_weight': 'balanced', 'model__C': 9}
Лучшая оценка: 0.7682465571259567


Логистическая регрессия не справилась с минимальным порогом в 0.75

In [14]:
f1_scorer = make_scorer(f1_score)

In [33]:
%%time
model_forest = RandomForestClassifier(random_state=RS)
params = {
    'model__n_estimators': [40, 70, 110],
    'model__max_depth': [1, 9, 2],
    'model__min_samples_split': [2, 5, 10]
}

pipeline = Pipeline([
    ('vect', CountVectorizer(min_df=10, max_df=1.0, ngram_range=(1, 4))),
    ('tfidf', TfidfTransformer()),
    ('model', RandomForestClassifier(random_state=RS))
])

grid = GridSearchCV(pipeline, cv=3, n_jobs=-1, param_grid=params, verbose=3, scoring=f1_scorer)
grid.fit(x_train['text_lemmatized'], y_train)
grid.best_score_

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Wall time: 4min 14s


0.0014815729648383718

In [46]:
%%time

print("Best parameters: ", grid_forest.best_params_)
print("Best f1 score: ", grid_forest.best_score_)

Best parameters:  {'max_depth': 1, 'n_estimators': 40}
Best f1 score:  0.0
Wall time: 0 ns


In [47]:
%%time
model_lgb = lgb.LGBMClassifier()
param_grid_lgb = {
    'boosting_type': ['dart'],
    'num_leaves': [100,150],
    'learning_rate': np.arange(0.3, 0.4, 0.1),
    'n_estimators': [100, 150]}
grid_lgb = GridSearchCV(estimator=model_lgb, param_grid=param_grid_lgb, scoring=f1_scorer, cv=2, verbose=2)
grid_lgb.fit(train_tfidf, y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV] END boosting_type=dart, learning_rate=0.3, n_estimators=100, num_leaves=100; total time=  10.0s
[CV] END boosting_type=dart, learning_rate=0.3, n_estimators=100, num_leaves=100; total time=   9.9s
[CV] END boosting_type=dart, learning_rate=0.3, n_estimators=100, num_leaves=150; total time=  13.4s
[CV] END boosting_type=dart, learning_rate=0.3, n_estimators=100, num_leaves=150; total time=  13.4s
[CV] END boosting_type=dart, learning_rate=0.3, n_estimators=150, num_leaves=100; total time=  15.8s
[CV] END boosting_type=dart, learning_rate=0.3, n_estimators=150, num_leaves=100; total time=  15.6s
[CV] END boosting_type=dart, learning_rate=0.3, n_estimators=150, num_leaves=150; total time=  21.9s
[CV] END boosting_type=dart, learning_rate=0.3, n_estimators=150, num_leaves=150; total time=  21.7s
[CV] END boosting_type=dart, learning_rate=0.4, n_estimators=100, num_leaves=100; total time=  10.1s
[CV] END boosting_type=dart, le

GridSearchCV(cv=2, estimator=LGBMClassifier(),
             param_grid={'boosting_type': ['dart'],
                         'learning_rate': array([0.3, 0.4]),
                         'n_estimators': [100, 150], 'num_leaves': [100, 150]},
             scoring=make_scorer(f1_score), verbose=2)

In [48]:
print("Best parameters:", grid_lgb.best_params_)
print("Best f1 score lgb:", grid_lgb.best_score_)

Best parameters: {'boosting_type': 'dart', 'learning_rate': 0.4, 'n_estimators': 100, 'num_leaves': 100}
Best f1 score lgb: 0.7499807300227117


In [62]:
%%time
model_cat = CatBoostClassifier()
param_cat_grid = {'learning_rate': np.arange(0.1, 0.3, 0.1),
                 'depth': [4,6,8],
                 'iterations': [ 100, 500]}

grid_cat = GridSearchCV(estimator=model_cat, param_grid=param_cat_grid, cv=2, verbose=3, scoring=f1_scorer)
grid_cat.fit(train_tfidf, y_train)

Fitting 2 folds for each of 12 candidates, totalling 24 fits
0:	learn: 0.5916222	total: 156ms	remaining: 15.5s
1:	learn: 0.5171360	total: 305ms	remaining: 14.9s
2:	learn: 0.4585792	total: 460ms	remaining: 14.9s
3:	learn: 0.4127875	total: 616ms	remaining: 14.8s
4:	learn: 0.3761455	total: 768ms	remaining: 14.6s
5:	learn: 0.3493415	total: 917ms	remaining: 14.4s
6:	learn: 0.3275737	total: 1.07s	remaining: 14.2s
7:	learn: 0.3098335	total: 1.22s	remaining: 14s
8:	learn: 0.2957930	total: 1.37s	remaining: 13.9s
9:	learn: 0.2827186	total: 1.52s	remaining: 13.7s
10:	learn: 0.2736959	total: 1.68s	remaining: 13.6s
11:	learn: 0.2658746	total: 1.83s	remaining: 13.4s
12:	learn: 0.2591716	total: 1.99s	remaining: 13.3s
13:	learn: 0.2539045	total: 2.14s	remaining: 13.2s
14:	learn: 0.2495839	total: 2.29s	remaining: 13s
15:	learn: 0.2459928	total: 2.45s	remaining: 12.9s
16:	learn: 0.2426379	total: 2.61s	remaining: 12.7s
17:	learn: 0.2396345	total: 2.75s	remaining: 12.6s
18:	learn: 0.2371433	total: 2.91s	r

60:	learn: 0.1980529	total: 9.79s	remaining: 6.26s
61:	learn: 0.1976168	total: 9.95s	remaining: 6.09s
62:	learn: 0.1970925	total: 10.1s	remaining: 5.93s
63:	learn: 0.1965863	total: 10.3s	remaining: 5.77s
64:	learn: 0.1960934	total: 10.4s	remaining: 5.61s
65:	learn: 0.1954763	total: 10.6s	remaining: 5.46s
66:	learn: 0.1948567	total: 10.8s	remaining: 5.3s
67:	learn: 0.1944201	total: 10.9s	remaining: 5.14s
68:	learn: 0.1939349	total: 11.1s	remaining: 4.98s
69:	learn: 0.1933620	total: 11.2s	remaining: 4.82s
70:	learn: 0.1927247	total: 11.4s	remaining: 4.66s
71:	learn: 0.1922604	total: 11.6s	remaining: 4.5s
72:	learn: 0.1919032	total: 11.7s	remaining: 4.33s
73:	learn: 0.1914385	total: 11.9s	remaining: 4.17s
74:	learn: 0.1909064	total: 12s	remaining: 4.01s
75:	learn: 0.1904389	total: 12.2s	remaining: 3.85s
76:	learn: 0.1899424	total: 12.3s	remaining: 3.69s
77:	learn: 0.1892397	total: 12.5s	remaining: 3.53s
78:	learn: 0.1888668	total: 12.7s	remaining: 3.37s
79:	learn: 0.1882788	total: 12.8s	r

20:	learn: 0.2102232	total: 3.28s	remaining: 12.3s
21:	learn: 0.2081122	total: 3.44s	remaining: 12.2s
22:	learn: 0.2066961	total: 3.6s	remaining: 12s
23:	learn: 0.2054752	total: 3.75s	remaining: 11.9s
24:	learn: 0.2034804	total: 3.9s	remaining: 11.7s
25:	learn: 0.2019255	total: 4.05s	remaining: 11.5s
26:	learn: 0.2005848	total: 4.21s	remaining: 11.4s
27:	learn: 0.1992617	total: 4.36s	remaining: 11.2s
28:	learn: 0.1975328	total: 4.52s	remaining: 11.1s
29:	learn: 0.1965658	total: 4.66s	remaining: 10.9s
30:	learn: 0.1954436	total: 4.82s	remaining: 10.7s
31:	learn: 0.1944249	total: 4.96s	remaining: 10.6s
32:	learn: 0.1935559	total: 5.12s	remaining: 10.4s
33:	learn: 0.1923553	total: 5.28s	remaining: 10.2s
34:	learn: 0.1915877	total: 5.42s	remaining: 10.1s
35:	learn: 0.1907585	total: 5.57s	remaining: 9.91s
36:	learn: 0.1898363	total: 5.72s	remaining: 9.75s
37:	learn: 0.1891345	total: 5.88s	remaining: 9.59s
38:	learn: 0.1880695	total: 6.03s	remaining: 9.43s
39:	learn: 0.1872913	total: 6.18s	r

80:	learn: 0.1839571	total: 12.5s	remaining: 1m 4s
81:	learn: 0.1835324	total: 12.6s	remaining: 1m 4s
82:	learn: 0.1831835	total: 12.8s	remaining: 1m 4s
83:	learn: 0.1827040	total: 12.9s	remaining: 1m 4s
84:	learn: 0.1822049	total: 13.1s	remaining: 1m 3s
85:	learn: 0.1818852	total: 13.2s	remaining: 1m 3s
86:	learn: 0.1815524	total: 13.4s	remaining: 1m 3s
87:	learn: 0.1812464	total: 13.6s	remaining: 1m 3s
88:	learn: 0.1809094	total: 13.7s	remaining: 1m 3s
89:	learn: 0.1804753	total: 13.9s	remaining: 1m 3s
90:	learn: 0.1801305	total: 14s	remaining: 1m 3s
91:	learn: 0.1798124	total: 14.2s	remaining: 1m 2s
92:	learn: 0.1795369	total: 14.3s	remaining: 1m 2s
93:	learn: 0.1792549	total: 14.5s	remaining: 1m 2s
94:	learn: 0.1789440	total: 14.6s	remaining: 1m 2s
95:	learn: 0.1786529	total: 14.8s	remaining: 1m 2s
96:	learn: 0.1783066	total: 15s	remaining: 1m 2s
97:	learn: 0.1777632	total: 15.1s	remaining: 1m 2s
98:	learn: 0.1774862	total: 15.3s	remaining: 1m 1s
99:	learn: 0.1771531	total: 15.4s	r

240:	learn: 0.1471976	total: 37s	remaining: 39.7s
241:	learn: 0.1470374	total: 37.2s	remaining: 39.6s
242:	learn: 0.1468527	total: 37.3s	remaining: 39.5s
243:	learn: 0.1467449	total: 37.5s	remaining: 39.3s
244:	learn: 0.1465350	total: 37.6s	remaining: 39.2s
245:	learn: 0.1464634	total: 37.8s	remaining: 39s
246:	learn: 0.1463226	total: 37.9s	remaining: 38.8s
247:	learn: 0.1462085	total: 38.1s	remaining: 38.7s
248:	learn: 0.1459758	total: 38.2s	remaining: 38.5s
249:	learn: 0.1458508	total: 38.4s	remaining: 38.4s
250:	learn: 0.1457256	total: 38.5s	remaining: 38.2s
251:	learn: 0.1456073	total: 38.7s	remaining: 38.1s
252:	learn: 0.1454694	total: 38.9s	remaining: 37.9s
253:	learn: 0.1453694	total: 39s	remaining: 37.8s
254:	learn: 0.1452922	total: 39.2s	remaining: 37.6s
255:	learn: 0.1452171	total: 39.3s	remaining: 37.5s
256:	learn: 0.1450480	total: 39.5s	remaining: 37.3s
257:	learn: 0.1449010	total: 39.6s	remaining: 37.2s
258:	learn: 0.1447606	total: 39.8s	remaining: 37s
259:	learn: 0.144627

...

18:	learn: 0.1902659	total: 38.7s	remaining: 16m 19s
19:	learn: 0.1884579	total: 40.8s	remaining: 16m 18s
20:	learn: 0.1871432	total: 42.8s	remaining: 16m 16s
21:	learn: 0.1856703	total: 44.8s	remaining: 16m 14s
22:	learn: 0.1839542	total: 46.9s	remaining: 16m 12s
23:	learn: 0.1825499	total: 48.9s	remaining: 16m 9s
24:	learn: 0.1811147	total: 50.8s	remaining: 16m 5s
25:	learn: 0.1800847	total: 52.9s	remaining: 16m 3s
26:	learn: 0.1786290	total: 54.8s	remaining: 16m
27:	learn: 0.1774701	total: 56.9s	remaining: 15m 59s
28:	learn: 0.1763236	total: 59s	remaining: 15m 57s
29:	learn: 0.1751957	total: 1m 1s	remaining: 15m 56s
30:	learn: 0.1741154	total: 1m 3s	remaining: 15m 56s
31:	learn: 0.1732012	total: 1m 5s	remaining: 15m 57s
32:	learn: 0.1717874	total: 1m 7s	remaining: 15m 55s
33:	learn: 0.1710069	total: 1m 9s	remaining: 15m 53s
34:	learn: 0.1698742	total: 1m 11s	remaining: 15m 52s
35:	learn: 0.1687721	total: 1m 13s	remaining: 15m 49s
36:	learn: 0.1680535	total: 1m 15s	remaining: 15m 47s

170:	learn: 0.1172220	total: 5m 50s	remaining: 11m 15s
171:	learn: 0.1170080	total: 5m 52s	remaining: 11m 13s
172:	learn: 0.1168390	total: 5m 54s	remaining: 11m 10s
173:	learn: 0.1167421	total: 5m 56s	remaining: 11m 8s
174:	learn: 0.1165115	total: 5m 58s	remaining: 11m 6s
175:	learn: 0.1164212	total: 6m	remaining: 11m 4s
176:	learn: 0.1161459	total: 6m 2s	remaining: 11m 2s
177:	learn: 0.1159937	total: 6m 4s	remaining: 11m
178:	learn: 0.1158518	total: 6m 6s	remaining: 10m 58s
179:	learn: 0.1156226	total: 6m 8s	remaining: 10m 55s
180:	learn: 0.1154042	total: 6m 10s	remaining: 10m 53s
181:	learn: 0.1150879	total: 6m 12s	remaining: 10m 51s
182:	learn: 0.1148743	total: 6m 14s	remaining: 10m 49s
183:	learn: 0.1146803	total: 6m 16s	remaining: 10m 47s
184:	learn: 0.1144455	total: 6m 19s	remaining: 10m 45s
185:	learn: 0.1143560	total: 6m 21s	remaining: 10m 43s
186:	learn: 0.1141265	total: 6m 23s	remaining: 10m 41s
187:	learn: 0.1140309	total: 6m 25s	remaining: 10m 39s
188:	learn: 0.1139438	tota

322:	learn: 0.0972434	total: 11m 9s	remaining: 6m 6s
323:	learn: 0.0971937	total: 11m 11s	remaining: 6m 4s
324:	learn: 0.0971436	total: 11m 13s	remaining: 6m 2s
325:	learn: 0.0970938	total: 11m 15s	remaining: 6m
326:	learn: 0.0970437	total: 11m 17s	remaining: 5m 58s
327:	learn: 0.0969937	total: 11m 19s	remaining: 5m 56s
328:	learn: 0.0969443	total: 11m 21s	remaining: 5m 54s
329:	learn: 0.0968822	total: 11m 23s	remaining: 5m 52s
330:	learn: 0.0968340	total: 11m 25s	remaining: 5m 50s
331:	learn: 0.0967434	total: 11m 28s	remaining: 5m 48s
332:	learn: 0.0965111	total: 11m 30s	remaining: 5m 46s
333:	learn: 0.0964623	total: 11m 32s	remaining: 5m 44s
334:	learn: 0.0964042	total: 11m 34s	remaining: 5m 41s
335:	learn: 0.0963552	total: 11m 36s	remaining: 5m 39s
336:	learn: 0.0962734	total: 11m 38s	remaining: 5m 37s
337:	learn: 0.0961414	total: 11m 40s	remaining: 5m 35s
338:	learn: 0.0960860	total: 11m 42s	remaining: 5m 33s
339:	learn: 0.0959111	total: 11m 44s	remaining: 5m 31s
340:	learn: 0.0957

473:	learn: 0.0871628	total: 16m 19s	remaining: 53.7s
474:	learn: 0.0871248	total: 16m 21s	remaining: 51.7s
475:	learn: 0.0870865	total: 16m 23s	remaining: 49.6s
476:	learn: 0.0870483	total: 16m 25s	remaining: 47.5s
477:	learn: 0.0869108	total: 16m 27s	remaining: 45.5s
478:	learn: 0.0868726	total: 16m 30s	remaining: 43.4s
479:	learn: 0.0868334	total: 16m 32s	remaining: 41.3s
480:	learn: 0.0867951	total: 16m 34s	remaining: 39.3s
481:	learn: 0.0867569	total: 16m 36s	remaining: 37.2s
482:	learn: 0.0867193	total: 16m 38s	remaining: 35.1s
483:	learn: 0.0866820	total: 16m 40s	remaining: 33.1s
484:	learn: 0.0866444	total: 16m 42s	remaining: 31s
485:	learn: 0.0865581	total: 16m 44s	remaining: 28.9s
486:	learn: 0.0865207	total: 16m 46s	remaining: 26.9s
487:	learn: 0.0864833	total: 16m 48s	remaining: 24.8s
488:	learn: 0.0864458	total: 16m 50s	remaining: 22.7s
489:	learn: 0.0864090	total: 16m 52s	remaining: 20.7s
490:	learn: 0.0863719	total: 16m 54s	remaining: 18.6s
491:	learn: 0.0863352	total: 1

GridSearchCV(cv=2,
             estimator=<catboost.core.CatBoostClassifier object at 0x000001829B2E2820>,
             param_grid={'depth': [4, 6, 8], 'iterations': [100, 500],
                         'learning_rate': array([0.1, 0.2])},
             scoring=make_scorer(f1_score), verbose=3)

In [63]:
%%time
best_cat = CatBoostClassifier(**grid_cat.best_params_)
best_cat.fit(train_tfidf, y_train)

y_cat_pred = best_cat.predict(valid_tfidf)
f1_cat = f1_score(y_valid, y_cat_pred)

print("Best parameters: ", grid_cat.best_params_)
print("Best RMSE score: ", grid_cat.best_score_)

0:	learn: 0.5170536	total: 1.74s	remaining: 14m 26s
1:	learn: 0.4052554	total: 3.8s	remaining: 15m 46s
2:	learn: 0.3344588	total: 5.88s	remaining: 16m 14s
3:	learn: 0.2933226	total: 8.08s	remaining: 16m 41s
4:	learn: 0.2666289	total: 10.2s	remaining: 16m 45s
5:	learn: 0.2495567	total: 12.3s	remaining: 16m 56s
6:	learn: 0.2376379	total: 14.5s	remaining: 17m 1s
7:	learn: 0.2270916	total: 16.6s	remaining: 17m
8:	learn: 0.2207240	total: 18.7s	remaining: 17m
9:	learn: 0.2154455	total: 20.9s	remaining: 17m 2s
10:	learn: 0.2116262	total: 22.9s	remaining: 16m 59s
11:	learn: 0.2081980	total: 25.1s	remaining: 16m 58s
12:	learn: 0.2055707	total: 27.2s	remaining: 17m
13:	learn: 0.2026354	total: 29.3s	remaining: 16m 58s
14:	learn: 0.1993634	total: 31.4s	remaining: 16m 54s
15:	learn: 0.1970646	total: 33.4s	remaining: 16m 49s
16:	learn: 0.1953072	total: 35.4s	remaining: 16m 45s
17:	learn: 0.1928250	total: 37.4s	remaining: 16m 42s
18:	learn: 0.1902659	total: 39.5s	remaining: 16m 40s
19:	learn: 0.18845

153:	learn: 0.1205172	total: 5m 17s	remaining: 11m 52s
154:	learn: 0.1201636	total: 5m 19s	remaining: 11m 50s
155:	learn: 0.1200122	total: 5m 21s	remaining: 11m 48s
156:	learn: 0.1198309	total: 5m 23s	remaining: 11m 46s
157:	learn: 0.1195530	total: 5m 25s	remaining: 11m 44s
158:	learn: 0.1194488	total: 5m 27s	remaining: 11m 42s
159:	learn: 0.1191564	total: 5m 29s	remaining: 11m 39s
160:	learn: 0.1190370	total: 5m 31s	remaining: 11m 37s
161:	learn: 0.1186898	total: 5m 33s	remaining: 11m 35s
162:	learn: 0.1185905	total: 5m 35s	remaining: 11m 33s
163:	learn: 0.1183739	total: 5m 37s	remaining: 11m 31s
164:	learn: 0.1182557	total: 5m 39s	remaining: 11m 29s
165:	learn: 0.1181539	total: 5m 41s	remaining: 11m 26s
166:	learn: 0.1180071	total: 5m 43s	remaining: 11m 24s
167:	learn: 0.1178955	total: 5m 45s	remaining: 11m 22s
168:	learn: 0.1176327	total: 5m 47s	remaining: 11m 20s
169:	learn: 0.1173612	total: 5m 49s	remaining: 11m 18s
170:	learn: 0.1172220	total: 5m 51s	remaining: 11m 16s
171:	learn

305:	learn: 0.0984979	total: 10m 22s	remaining: 6m 34s
306:	learn: 0.0983881	total: 10m 24s	remaining: 6m 32s
307:	learn: 0.0983182	total: 10m 26s	remaining: 6m 30s
308:	learn: 0.0982672	total: 10m 28s	remaining: 6m 28s
309:	learn: 0.0982178	total: 10m 30s	remaining: 6m 26s
310:	learn: 0.0981680	total: 10m 32s	remaining: 6m 24s
311:	learn: 0.0981180	total: 10m 34s	remaining: 6m 22s
312:	learn: 0.0980672	total: 10m 36s	remaining: 6m 20s
313:	learn: 0.0979330	total: 10m 38s	remaining: 6m 18s
314:	learn: 0.0978271	total: 10m 40s	remaining: 6m 15s
315:	learn: 0.0977775	total: 10m 42s	remaining: 6m 13s
316:	learn: 0.0975411	total: 10m 44s	remaining: 6m 11s
317:	learn: 0.0974905	total: 10m 46s	remaining: 6m 9s
318:	learn: 0.0974404	total: 10m 48s	remaining: 6m 7s
319:	learn: 0.0973925	total: 10m 50s	remaining: 6m 5s
320:	learn: 0.0973433	total: 10m 51s	remaining: 6m 3s
321:	learn: 0.0972931	total: 10m 53s	remaining: 6m 1s
322:	learn: 0.0972434	total: 10m 55s	remaining: 5m 59s
323:	learn: 0.0

456:	learn: 0.0881644	total: 15m 24s	remaining: 1m 27s
457:	learn: 0.0880364	total: 15m 26s	remaining: 1m 25s
458:	learn: 0.0879986	total: 15m 28s	remaining: 1m 22s
459:	learn: 0.0879035	total: 15m 30s	remaining: 1m 20s
460:	learn: 0.0878649	total: 15m 32s	remaining: 1m 18s
461:	learn: 0.0878259	total: 15m 34s	remaining: 1m 16s
462:	learn: 0.0877877	total: 15m 36s	remaining: 1m 14s
463:	learn: 0.0877489	total: 15m 38s	remaining: 1m 12s
464:	learn: 0.0876609	total: 15m 40s	remaining: 1m 10s
465:	learn: 0.0875299	total: 15m 42s	remaining: 1m 8s
466:	learn: 0.0874909	total: 15m 44s	remaining: 1m 6s
467:	learn: 0.0874316	total: 15m 46s	remaining: 1m 4s
468:	learn: 0.0873932	total: 15m 48s	remaining: 1m 2s
469:	learn: 0.0873550	total: 15m 50s	remaining: 1m
470:	learn: 0.0872771	total: 15m 52s	remaining: 58.7s
471:	learn: 0.0872391	total: 15m 54s	remaining: 56.6s
472:	learn: 0.0872008	total: 15m 56s	remaining: 54.6s
473:	learn: 0.0871628	total: 15m 58s	remaining: 52.6s
474:	learn: 0.0871248	

In [64]:
x_features = vstack((train_tfidf, valid_tfidf))
y_features = pd.concat([y_train, y_valid], axis=0)

In [69]:
%%time
test_lgb = lgb.LGBMClassifier(**grid_lgb.best_params_)
test_lgb.fit(x_features, y_features)

y_test_pred = test_lgb.predict(test_tfidf)
f1_lgb = f1_score(y_test, y_test_pred)
print("Best f1 score на тестовой выборке: ", f1_test)

Best f1 score на тестовой выборке:  0.7801671003972058
Wall time: 32.2 s


In [66]:
model = DummyClassifier(strategy = 'constant', constant = 1)
model.fit(train_tfidf, y_train)
y_pred = model.predict(test_tfidf)
f1_dum = f1_score(y_test, y_pred)
print("F1 score: ", f1_dum)

F1 score:  0.18449965808069296


In [70]:
df_pred = pd.DataFrame({'predict': f1_lgb, 'true': y_test})
df_pred['predict'] = df_pred['predict'].astype(int)
df_pred.sample(15)

,predict,true
29533,0,0
45806,0,0
94629,0,0
18711,0,0
42211,0,0
134621,0,0
41872,0,0
108871,0,0
59507,0,0
39790,0,0


Модель значительно лучше константной. CatBoost занимает кучу времени, поэтому побеждает lgb.

# Вывод
1. Данные загружены.
2. Текст лемматизирован.
3. Применен метод tf-idf.
4. Модели обучены на полученной матрице.
5. Модели градиентного бустинга прошли порог f1 меры в 0.75 и показали результат 0.78
6. Модель сильно лучше константной и готова к работе.